In [ ]:
import os

if not os.getcwd().endswith("src"):
    %cd ..
%pwd

/home/xqz-u/FACT/src


'/home/xqz-u/FACT/src'

In [ ]:
import functools as ft
import itertools as it
import logging
import pprint
from copy import deepcopy
from typing import Dict, Sequence, Tuple

import implicit
import numpy as np
from implicit import evaluation
from scipy import sparse
import wandb

import config
import constants
import datasets
import utils

logger = logging.getLogger(__name__)
utils.setup_root_logging()
logger.info("Implicit using GPU: %s", implicit.gpu.HAS_CUDA)

p19368 [23-01-19 12:24] [INFO]  Implicit using GPU: True  (__main__:21)


In [ ]:
# metric = "map"
# metric = "ndcg"
# metric = "precision"
metric = "auc"

# Define sweep config
sweep_configuration = {
    'method': 'grid',
    'name': f'sweep_lastfm_ground_truth_{metric}',
    'metric': {'goal': 'maximize', 'name': metric},
    'parameters': {k: {"values": v} for k, v in constants.ground_truth_hparams.items()}
}
pprint.pprint(sweep_configuration)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'auc'},
 'name': 'sweep_lastfm_ground_truth_auc',
 'parameters': {'alpha': {'values': [0.1, 1, 10, 100]},
                'factors': {'values': [16, 32, 64, 128]},
                'regularization': {'values': [0.01, 0.1, 1, 10]}}}


In [ ]:
def recommender_grid_search(
    train_mat: sparse.csr_matrix,
    valid_mat: sparse.csr_matrix,
    metric: str
) -> implicit.als.AlternatingLeastSquares:
    
    run = wandb.init(dir=config.WANDB_DIR)
    hparams = wandb.config
    
    logger.info("Metric: %s, hyperparameters:", metric)
    logger.info(pprint.pformat(hparams))

    model = implicit.als.AlternatingLeastSquares(**hparams)
    model.fit(train_mat)
    metrics = evaluation.ranking_metrics_at_k(model, train_mat, valid_mat)
    score = metrics[metric]
    wandb.log({**hparams, metric: score})

    # TODO save model! either in wandb or locally
    
    
def split_csr(mat_csr: sparse.csr_matrix, seed: int) -> Tuple[sparse.csr_matrix, ...]:
    # split into 0.7 train 0.2 val 0.1 test
    train_csr, tmp_csr = implicit.evaluation.train_test_split(
        mat_csr, train_percentage=0.7, random_state=seed
    )
    valid_csr, test_csr = implicit.evaluation.train_test_split(
        tmp_csr, train_percentage=2 / 3, random_state=seed
    )
    return train_csr, valid_csr, test_csr

In [ ]:
lastfm = datasets.get_lastfm()
lastfm.head()

p19368 [23-01-19 12:24] [INFO]  ../data/Lastfm already exists, abort download  (datasets:18)


item,2,6,7,8,9,10,12,15,18,19,...,18125,18126,18127,18205,18206,18434,18435,18558,18559,18575
user,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
lastfm_csr = sparse.csr_matrix(lastfm.values)
gt_train_csr, gt_valid_csr, _ = split_csr(lastfm_csr, constants.SEED)
gt_train_csr, gt_valid_csr

(<1892x2500 sparse matrix of type '<class 'numpy.float64'>'
 	with 47004 stored elements in Compressed Sparse Row format>,
 <1892x2500 sparse matrix of type '<class 'numpy.float64'>'
 	with 13362 stored elements in Compressed Sparse Row format>)

In [ ]:
# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project='recommender', entity="fact-ai")
sweep_id

p19368 [23-01-19 12:24] [ERROR]  Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.  (wandb.jupyter:232)


Create sweep with ID: pngs1wi5
Sweep URL: https://wandb.ai/fact-ai/recommender/sweeps/pngs1wi5


'pngs1wi5'

In [ ]:
# Start sweep job.
wandb.agent(sweep_id, function=ft.partial(recommender_grid_search, gt_train_csr, gt_valid_csr, metric))

p19368 [23-01-19 12:24] [INFO]  Starting sweep agent: entity=None, project=None, count=None  (wandb.agents.pyagent:314)
wandb: Agent Starting Run: q92foirq with config:
wandb: 	alpha: 0.1
wandb: 	factors: 16
wandb: 	regularization: 0.01
p19368 [23-01-19 12:24] [ERROR]  Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.  (wandb.jupyter:232)
wandb: Currently logged in as: xqz-u (fact-ai). Use `wandb login --relogin` to force relogin


p19368 [23-01-19 12:24] [INFO]  Metric: auc, hyperparameters:  (__main__:10)
p19368 [23-01-19 12:24] [INFO]  {'alpha': 0.1, 'factors': 16, 'regularization': 0.01}  (__main__:11)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1839 [00:00<?, ?it/s]

alpha,▁
auc,▁
factors,▁
regularization,▁
alpha,0.1
auc,0.60874
factors,16
regularization,0.01


...

p19368 [23-01-19 12:26] [INFO]  Metric: auc, hyperparameters:  (__main__:10)
p19368 [23-01-19 12:26] [INFO]  {'alpha': 0.1, 'factors': 64, 'regularization': 0.01}  (__main__:11)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1839 [00:00<?, ?it/s]

alpha,▁
auc,▁
factors,▁
regularization,▁
alpha,0.1
auc,0.60535
factors,64
regularization,0.01


wandb: Agent Starting Run: y4x26b2u with config:
wandb: 	alpha: 0.1
wandb: 	factors: 64
wandb: 	regularization: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [ ]:
# ground_truth_paths = [
#     # config.MODELS_DIR / "model_lastfm_ground_truth.npz",
#     # config.MODELS_DIR / "hparams_lastfm_ground_truth.txt",
#     "model_lastfm_ground_truth.npz",
#     "hparams_lastfm_ground_truth.txt"
# ]
# 
# hparams = {'factors': [16, ],
#  'regularization': [0.01, 0.1],
#  'alpha': [0.1]}
# 
# gt_best_model = recommender_grid_search(gt_train_csr, gt_valid_csr, ground_truth_paths, "lastfm_ground_truth", 
#                                         #**hparams
#                                         **constants.ground_truth_hparams
#                                        )

In [ ]:
# def train_eval_recommender(
#     train_mat: sparse.csr_matrix,
#     valid_mat: sparse.csr_matrix,
#     **hparams
# ) -> Tuple[Dict[str, float], implicit.als.AlternatingLeastSquares]:
#     model = implicit.als.AlternatingLeastSquares(**hparams)
#     model.fit(train_mat)
#     metrics = evaluation.ranking_metrics_at_k(model, train_mat, valid_mat)
#     wandb.log(metrics)
#     return metrics, model
#def recommender_grid_search(
#    train_mat: sparse.csr_matrix,
#    valid_mat: sparse.csr_matrix,
#    best_model_paths: Sequence[str],
#    run_name: str,
#    metric: str = "map",
#    **hyperparams: Dict[str, Sequence[float]],
#) -> implicit.als.AlternatingLeastSquares:
#    
#    logger.info("Hyperparameters in grid search:")
#    logger.info(pprint.pformat(hyperparams))
#    hyperparams_flat = list(it.product(*hyperparams.values()))
#
#    best_model_path, best_hparams_path = best_model_paths
#    best_score, best_model, best_hparams = -1.0, None, None
#
#    for i, hparams in enumerate(hyperparams_flat):
#        hparams_dict = dict(zip(hyperparams.keys(), hparams))
#        run = wandb.init(project="recommender", entity="fact-ai", name=f"{run_name}_{i}", dir=config.WANDB_DIR, config=hparams_dict, reinit=True)
#        with run:
#            metrics, model = train_eval_recommender(train_mat, valid_mat, **hparams_dict)
#        score = metrics[metric]
#        if score > best_score:
#            logger.info(
#                "%d: Best model found! Old %s: %f new %s: %f hparams: %s",
#                i,
#                metric,
#                best_score,
#                metric,
#                score,
#                hparams,
#            )
#            best_score = score
#            best_model = deepcopy(model)
#            best_hparams = deepcopy(hparams)
#
#    best_model.save(best_model_path)
#    logger.debug("Saved best model to %s", best_model_path)
#    with open(best_hparams_path, "w") as fd:
#        fd.write("factor,regularizer,alpha,metric,score\n")
#        fd.write(f"{','.join(list(map(str, best_hparams + (metric, best_score))))}\n")
#    logger.debug("Saved best model hyperparams to %s", best_hparams_path)
#    return best_model